# Azure ML Deployment

In [ ]:
!pip install azureml-core azureml-sdk -q

In [ ]:

from azureml.core import Workspace
ws = Workspace.from_config()
print(f"Connected to workspace: {ws.name}")


In [ ]:

from azureml.core.model import Model

model = Model.register(
    model_path="models/final_model.pkl",
    model_name="nlp-text-classifier",
    workspace=ws
)
print(f"Registered model: {model.name}")


In [ ]:

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification(name="nlp-env", file_path="environment.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=env)


In [ ]:

from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name="nlp-api-service",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print(f"Deployed at: {service.scoring_uri}")
